In [ ]:
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from IPython.display import display
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_curve, auc,
)

In [ ]:
sns.set_theme(style='whitegrid')

### Metrics calculation

In [ ]:
with open('./data/experiments/config.json') as fp:
    config = json.load(fp)

slides = {
    'amy': config['shards']['amy'] + config['testing'],
    'ben': config['shards']['ben'] + config['testing'],
    'charlie': config['shards']['charlie'] + config['testing'],
    'ensemble': config['testing'],
}

In [ ]:
def process_single(slide: str, model: str, size: int) -> pd.Series:
    data = pd.read_parquet(f'data/results/{slide}/predictions-f{size}-s16-m0-{model}.gz.parquet')
    y_true = data.labels.astype(float).round().astype(bool)
    y_pred = data.prediction.astype(float).round().astype(bool)
    return pd.Series(dict(
        slide=slide,
        fragment=size,
        model=model,
        accuracy=accuracy_score(y_true, y_pred) * 100,
        recall=recall_score(y_true, y_pred, zero_division=0) * 100,
        precision=precision_score(y_true, y_pred, zero_division=0) * 100,
        f1=f1_score(y_true, y_pred, zero_division=0) * 100,
    ))

def process_slide(slide: str, model: str) -> pd.DataFrame:
    return pd.DataFrame([
        process_single(slide, model, size)
        for size in [48, 96, 192]
    ])

In [ ]:
data = pd.concat([
    process_slide(slide, model)
    for slide, model in tqdm([
        (slide, model)
        for model in slides.keys()
        for slide in slides[model]
    ])],
    ignore_index=True,
)
data.to_csv('data/experiments/cnn.csv', index=False)

In [ ]:
data = pd.read_csv('data/experiments/cnn.csv')
data

### Results

In [ ]:
display(
    data[~data.slide.isin(slides['ensemble'])]
    .groupby('fragment')
    .mean()
)

In [ ]:
df_single = (
    data[data.slide.isin(slides['ensemble']) & (data.model != 'ensemble')]
    .groupby(['slide', 'fragment'])
    .mean()
    .reset_index()
    .groupby('fragment')
    .mean()
)
df_ensemble = (
    data[data.slide.isin(slides['ensemble']) & (data.model == 'ensemble')]
    .groupby('fragment')
    .mean()
)

df = pd.concat([df_single, df_ensemble], keys=['single', 'ensmble'])
display(df)

### Plots

In [ ]:
df_single = (
    data[data.slide.isin(slides['ensemble']) & (data.model != 'ensemble')]
    .groupby(['slide', 'fragment'])
    .mean()
    .reset_index()
    .assign(Model='Single')
)
df_ensemble = (
    data[data.slide.isin(slides['ensemble']) & (data.model == 'ensemble')]
    .assign(Model='Ensemble')
 )

df = pd.concat([df_single, df_ensemble], ignore_index=True)

fig, ax = plt.subplots(figsize=(8, 4))
g = sns.boxplot(x='fragment', y='f1', hue='Model', data=df, showfliers=False, ax=ax)
g.set_xlabel('Window size')
g.set_xticklabels(['48 px', '96 px', '192 px'])
g.set_ylabel('F1-score [%]')
None

In [ ]:
predictions = pd.concat([
    pd.read_parquet(f'data/results/{slide}/predictions-f{size}-s16-m0-ensemble.gz.parquet').assign(fragment=size)[['labels', 'amy', 'ben', 'charlie', 'fragment']]
    for slide in slides['ensemble']
    for size in [48, 96, 192]
], ignore_index=True).astype(np.float32)
df = pd.melt(predictions, id_vars=('labels', 'fragment'), value_vars=('amy', 'ben', 'charlie'))

fig, ax = plt.subplots(figsize=(4, 4))
for size in [48, 96, 192]:
    fpr, tpr, _ = roc_curve(
        df[df.fragment == size].labels.ravel(),
        df[df.fragment == size].value.ravel(),
        pos_label=1,
        drop_intermediate=False,
    )
    roc_auc = auc(fpr, tpr)
    ax.plot(fpr, tpr, lw=2, label=f'{size}px window (AUC = {roc_auc:.2f})')

ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.legend(loc="lower right")
None